In [5]:
from bs4 import BeautifulSoup 
import requests
import pickle 
import pandas as pd

In [6]:
# Assign URL 
url = "https://asn.flightsafety.org/database/engines/"

# Make a GET request to fetch the raw HTML content 
html_content = requests.get(url).text 


In [7]:
# Parse the response with html.parser
soup = BeautifulSoup(html_content,"html.parser") 


In [8]:
# The data we need are wrapped in <a> tags. Grab all the <a> tags here and we'll filter later.
datas = soup.find_all("a")

In [9]:
#Make list of URLs and list oF Aircraft to create dictionary
k = []
v = []
for item in datas:
    if 'engine' in item['href']:
        k.append(item['href'])
        v.append(item.text)

lookup_dict = {'URL': k, 'Engine': v}

In [10]:
url_df = pd.DataFrame.from_dict(lookup_dict)

In [11]:
with open('lookup_dict.pkl', 'wb') as f:
    pickle.dump(lookup_dict, f)

In [ ]:
#with open('zippedData/lookup_dict.pkl', 'rb') as f:
    #lookup_dict = pickle.load(f)

In [12]:
# Take in the engine and retriev the matching data page with Aircraft data
# Add Aircraft list from web a python list.
# Return the python list
url_p1 = 'https://asn.flightsafety.org'
def lookup_ac(url_p2):
    return_list = []
    url = f'{url_p1}{url_p2}'
    sub_content = requests.get(url).text
    soup2 = BeautifulSoup(sub_content,"html.parser")
    u_list=soup2.find_all('ul')[1]
    i = 0 
    while i < len(u_list.select('li')):
        (u_list.select('li')[i].text)
        return_list.append(u_list.select('li')[i].text)
        i += 1
    return(return_list)

In [13]:
# Create Aircraft column to list aircraft for each engine.
url_df['Aircraft'] = url_df['URL'].map(lookup_ac)

In [15]:
# Save df to pickle file.
url_df.to_pickle('zippedData/url_df.pkl')

In [16]:
# Create a row for each aircraft for each engine. 
# Iterate through url_df['Aircraft'] column. For each element
# in list, create a new row.

engines = []
aircraft = []
for idx, row in url_df.iterrows():
    for ac in row['Aircraft']:
        engines.append(row['Engine'])
        aircraft.append(ac)
        


In [17]:
ac_eng_dict = {'Engine': engines, 'Aircraft': aircraft}

In [18]:
engines_df = pd.DataFrame.from_dict(ac_eng_dict)

In [20]:
engines_df['Aircraft'].replace('-', '', inplace=True, regex=True)
engines_df['Aircraft'].replace('/', '', inplace=True, regex=True)

In [21]:
engines_df.drop_duplicates(inplace=True)

In [22]:
def get_short_name(model):
    m_string = model.split(' ')
    if len(m_string) == 1:
        return(m_string[0])
    else:
        return(m_string[1])
    

In [ ]:
''' 
This function will 
1) take in a model string from bts_map_df 
2) match it with a newmodel from engines_df
3) update engines_df with new_model column
TODO: Replace with faster version.
'''
def match_types(model):
    for newmodel in engines_df['Model']:
        if model.startswith(newmodel):
            #print(f'Processing: {model} \t:{newmodel}')
            return newmodel

In [23]:
# Read data from Aircraft_Safety dataset
bts_map_df = pd.read_excel('zippedData/bts_actype.2.xlsx')


In [24]:
bts_map_df

,Unnamed: 0,Description,Model
0,0,Boeing 757-200,757
1,72814,Pilatus PC-12,PC12
2,98137,Cessna Citation X Model 650/550B/550XL,CitationX
3,99813,Boeing 737-400,737
4,120190,Boeing 737-300,737
...,...,...,...
182,2018715,Boeing 747-200/300,747
183,2018731,Cessna 180A/B,180
184,2018733,Gulfstream Aerospace G500C,G500
185,2018734,Gulfstream Aerospace Gulfstream G600,G500


In [26]:
engines_df['Model'] = [get_short_name(model) for model in engines_df['Aircraft']]

In [27]:
engines_df.columns

Index(['Engine', 'Aircraft', 'Model'], dtype='object')

In [28]:
engines_df

,Engine,Aircraft,Model
0,Allison 250,BrittenNorman BN2 Islander,BN2
1,Allison 250,GAF Nomad,Nomad
2,Allison 501,Aero Spacelines Mini Guppy Turbine,Spacelines
3,Allison 501,Convair CV580,CV580
4,Allison 501,Lockheed C130 Hercules,C130
...,...,...,...
461,Wright R-3350,Lockheed P2 Neptune,P2
462,Wright R-3350,Martin Mars,Mars
463,Wright R-3350,Martin P5M Marlin,P5M
464,Wright R-975,Ford TriMotor,TriMotor
